# Substitute Dropped Frames with Synthetic Frames

-----
## Motivation

Due to hardware/software limitations and I/O speeds, sometimes footage from an experimental recording may have missing frames. 
This data must undergo preprocessing before routine analysis is carried out. Some reasons why we might want to do this are:
  1.  Different kinds of footage need to be synchonised , e.g. Behaviour and Brain Imaging footage from one or more mice, of which a combination may have missing frames
  2.  External libraries used for analysis require the data to have the same dimensions for some operations


## Proposed Solution: Interpolation

Suppose we have two known points and would like to know the value of a point smewhere in between these points. 
Consider the example below, were we would like to know the value of $y$ at $x=0.5$

<img src='images/1.jpeg'  style='display: block; margin-left: auto; margin-right: auto'>

In this case, we shall perform a linear interpolation.
Mathematically, it is defined as:

<center>$y = y_1 + (x - x_1) \frac{y_2 - y_1}{x_2 - x_1}$</center>

Where: $(x_1, y_1) = (0,3)$, $(x_2, y_2) = (1,5)$


Plugging this into the equation above, we have:

<center>$y = 3 + (0.5 - 0) \frac{5 - 3}{1 - 0} = 4$</center>

Plotting the point $(0.5,4)$, we can see that it does indeed lie on the line

<img src='images/2.jpeg'  style='display: block; margin-left: auto; margin-right: auto'>

This can be extended to video footage by taking the temporal evolution of each pixel in a frame as a 'line' like we did above

___

### Breaking down the problem

#### 1. Finding where we dropped the frames, and how many we dropped

Now that we know how we will generate the synthetic frames, we need to figure out how we detect and find the locations of missing frames.

Since the experimenter thought ahead, there is a text dump of the timestamps at which frames were written to the footage.

If one or more frames is missing between some time stamps, the difference between then will be greater than the regular period, which we can calculate if we know the frame rate the footage was recorded at.

### <center>$Period = \frac{1}{Frame Rate}$</center>

In [159]:
import numpy as np
from random import randint

In [178]:
period = 1_000_000/30  # 1e6 microseconds divided by 30 frames per second
threshold = period * 1.1

In [198]:
# load the timestamps
timestamps = np.fromfile('timestamps.txt', sep=',').astype('int')

# find the differences between the timestamps
differences = np.diff(timestamps)

# find out where the frames were dropped, and how mnay were dropped
locations = np.where(differences > threshold)[0]
print(f"Frames were dropped at {len(locations)} locations")

Frames were dropped at 11 locations


Wait! We are not quite done yet. What if multiple frames are dropped at one location? We can find out by looking at how large the `differences` are at `locations`

In [199]:
dropped_frames = []
for location in locations:
    number_dropped_frames = round(differences[location]/period) / 2
    dropped_frames.append(int(number_dropped_frames))

In [201]:
for location, num_dropped_frames in zip(locations, dropped_frames):
    print(f'{num_dropped_frames} frame(s) were dropped at index {location}')

1 frame(s) were dropped at index 32
1 frame(s) were dropped at index 241
1 frame(s) were dropped at index 795
1 frame(s) were dropped at index 1288
1 frame(s) were dropped at index 1691
1 frame(s) were dropped at index 2074
1 frame(s) were dropped at index 2089
1 frame(s) were dropped at index 2246
1 frame(s) were dropped at index 2617
1 frame(s) were dropped at index 2619
1 frame(s) were dropped at index 2670


#### 2. Generating the synthetic frames

Now that we know where the frames were dropped and how many, we can find the closest existing frames before and after these dropped frames and use these to generate the synthetic frames

In [179]:
# load the frames
frames = np.load('video.raw')

##### 2.1 We want to implement the formula for linear interpolation to generate synthetic frames:
<center>$y = y_1 + (x - x_1) \frac{y_2 - y_1}{x_2 - x_1}$</center>

Consider the following algorithm:
##### <center>for $i=1..n$ dropped frames:</center>
##### <center>$i = first frame + (last frame-first frame)\frac{i}{n+1}$</center>
##### <center>end for</center>
There are two immediate solutions that present themselves:
A. We could loop through each pixel in Python and construct the synthetic image
B. We could do in in NumPy

Let's do B. - it will be simpler to implement and more computationally efficient.
The corresponding Python function definition is:

In [213]:
def generate_synthetic_frames(first_frame, last_frame, num_dropped_frames):
    synthetic_frames = []
    for i in range(1, n_dropped_frames+1):  # to account for 0-based indexing in `range`
        synthetic_frame = first_frame + (last_frame-first_frame)*i/(num_dropped_frames+1)
        synthetic_frames.append(synthetic_frame)
    return np.asarray(synthetic_frames)

Let's run a few simple tests:

In [214]:
first_frame = np.ones([100,100])
last_frame = np.ones([100,100])*3
n_dropped_frames = 1

synthetic_frames = generate_synthetic_frames(first_frame, last_frame, n_dropped_frames)
assert((synthetic_frames[0] == (np.ones([100,100])*2)).all())
print('Test 1 Passed!')

Test 1 Passed!


In [215]:
first_frame = np.ones([100,100])
last_frame = np.ones([100,100])*4
n_dropped_frames = 2

synthetic_frames = generate_synthetic_frames(first_frame, last_frame, n_dropped_frames)
assert((synthetic_frames[0] == (np.ones([100,100])*2)).all())
assert((synthetic_frames[1] == (np.ones([100,100])*3)).all())
print('Test 2 Passed!')

Test 2 Passed!


Now we can generate all the synthetic frames required:

In [ ]:
listof_synthetic_frames = []
for location, num_dropped_frames in zip(locations, dropped_frames):
    synthetic_frames = generate_synthetic_frames(
        frames[location],
        frames[location+1],
        num_fropped_frames
    )
    listof_synthetic_frames.append(synthetic_frames)

##### 2.2 Inserting the synthetic frames into their correct locations 

As we insert the synthetic frames, the indeces we used to indentify the `locations` of the dropped frames will have to shift by the number of frames inserted each time we add some frames. Consider the following peudocode:
``` pseudocode
index_shift = 1
for location in locations:
    frames.insert(synthetic frames corresponding to location, at index location+index_shift)
    index_shift = index_shift + number of synthetic frames inserted
end for
```

The corresponding Python function definition is:

In [244]:
def insert_synthetic_frames(frames, locations, listof_synthetic_frames):
    index_shift = 1  # account for `locations` referring to an index deficient by one
    for location, synthetic_frames in zip(locations, listof_synthetic_frames):
        frames = np.insert(frames,  location+index_shift, synthetic_frames,0)
        index_shift += synthetic_frames.shape[0]
    return frames

Let's run some more tests:

In [258]:
test_frames = np.array([first_frame, last_frame])
test_locations = [0]
test_synthetic_frames = [generate_synthetic_frames(first_frame,last_frame,2)]
test_frames = insert_synthetic_frames(test_frames, test_locations, test_synthetic_frames)

for i in range(3):
    assert((test_frames[i] == (np.ones([100,100])*(i+1))).all())
print('Test 3 Passed!')

Test 3 Passed!


In [259]:
test_frames = np.array([np.ones([100,100]),
                        np.ones([100,100])*4,
                        np.ones([100,100])*5,
                        np.ones([100,100])*7
                       ])
test_locations = [0, 2]
test_num_dropped_frames = [2,1]
test_synthetic_frames = []
for location, num_dropped_frames in zip(test_locations, test_num_dropped_frames):
    synthetic_frames = generate_synthetic_frames(
        test_frames[location],
        test_frames[location+1],
        num_dropped_frames
    )
    test_synthetic_frames.append(synthetic_frames)
test_frames = insert_synthetic_frames(test_frames, test_locations, test_synthetic_frames)

for i in range(7):
    assert((test_frames[i] == (np.ones([100,100])*(i+1))).all())

print('Test 4 Passed!')

Test 4 Passed!


---

We now have all the machinery we need to fill the frames with our synthetic frames

In [ ]:
# Fill in the frames with the synthetic frames
frames = insert_synthetic_frames(frames, locations, listof_synthetic_frames)
# Save the new set of frames
np.save('video_filled_frames.raw', frames)